In [1]:
# !pip3 install openpyxl
import pandas as pd
import re


In [2]:
df = pd.read_excel(r'C:\Users\john_\Desktop\UN-project\UPR_cycle2.xlsx')

In [3]:
df.index+=1
df.rename(columns={"Reccomending Body": "Recommending Body", "UPR Reccomending States": "UPR Recommending States"}, inplace=True)
df.head(2)

,Text,Countries Concerned,Recommending Body,Document Symbol,Themes,Affected Persons,Sdgs,Document Publication Date,UPR Recommending States,UPR Position,Type,OHCHR Annotation Id,UPR Session,Regions Concerned,Recommending Regions,Date of publication on UHRI
1,117.22 Promptly ratify the Optional Protocol t...,- Uganda,- UPR,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Indigenous peoples,NaN,2016-12-27,- Guatemala,- Noted,- Recommendations,4100eed4-fcf7-42d6-941a-a89493e77e6c,- 26th Session - Cycle 2,- Africa,- Latin America and the Caribbean,2018-06-20 13:50:21.503
2,133.247 Ensure full and equal access to modern...,- Venezuela (Bolivarian Republic of),- UPR,A/HRC/34/6,- Sexual & reproductive health and rights,- Indigenous peoples\n- Persons living in rura...,NaN,2016-12-27,- Denmark,- Supported,- Recommendations,f910ce5d-0d1c-45de-b64b-3050df7679dc,- 26th Session - Cycle 2,- Latin America and the Caribbean,- Western Europe & Others,2018-05-04 12:18:58.713


# Data Cleaning

##### - Check unique number of text & annotation id to establish primary key
##### - Both have 625 unique entries which is the same as the total database entries
##### - Normalise by removing one of these, two identifiers is uneccesary we are not gaining anymore information

In [4]:
text_count = df.Text.unique().size
print(text_count)

625


In [5]:
annotation_id_count = df["OHCHR Annotation Id"].unique().size
print(annotation_id_count)

625


##### Columns to be dropped
##### Annotation ID, Recommending body, Text, Affected persons, Sdgs, Document publication date, session, date
##### check all entries are recommendation by seeing if text starts with number, then drop this column. 

In [6]:
drop_columns = ["OHCHR Annotation Id","Recommending Body", "Document Publication Date", "Sdgs", "UPR Session", "Date of publication on UHRI", "Affected Persons"]

In [7]:
df.drop(drop_columns, inplace=True, axis=1)

In [8]:
df.head(2)

,Text,Countries Concerned,Document Symbol,Themes,UPR Recommending States,UPR Position,Type,Regions Concerned,Recommending Regions
1,117.22 Promptly ratify the Optional Protocol t...,- Uganda,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Guatemala,- Noted,- Recommendations,- Africa,- Latin America and the Caribbean
2,133.247 Ensure full and equal access to modern...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Sexual & reproductive health and rights,- Denmark,- Supported,- Recommendations,- Latin America and the Caribbean,- Western Europe & Others


In [9]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [10]:
df = df[df['Text'].apply(lambda x: has_numbers(x))]
df.drop(['Type'], inplace=True, axis=1)

In [11]:
df.index = df.index.set_names(['Recommendation_id'])
df.head(2)

,Text,Countries Concerned,Document Symbol,Themes,UPR Recommending States,UPR Position,Regions Concerned,Recommending Regions
Recommendation_id,,,,,,,,
1,117.22 Promptly ratify the Optional Protocol t...,- Uganda,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Guatemala,- Noted,- Africa,- Latin America and the Caribbean
2,133.247 Ensure full and equal access to modern...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Sexual & reproductive health and rights,- Denmark,- Supported,- Latin America and the Caribbean,- Western Europe & Others


#### Standardize the naming convention, by removing hyphens and spaces, of all data entries.

In [12]:
def remove_leading_char(data_entry):
    """"Clean the naming convention for data entries of all columns"""
    clean_aff = re.sub(r'^[^A-Z]*', '', data_entry)
    if clean_aff:
        return clean_aff

In [13]:
def add_empty_cell_string(df):
    """Replacing all empty cells with default string 'Missing data'"""
    df.fillna('Missing data', inplace=True)
    return df 

In [14]:
df2 = add_empty_cell_string(df)
df2.head()

,Text,Countries Concerned,Document Symbol,Themes,UPR Recommending States,UPR Position,Regions Concerned,Recommending Regions
Recommendation_id,,,,,,,,
1,117.22 Promptly ratify the Optional Protocol t...,- Uganda,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Guatemala,- Noted,- Africa,- Latin America and the Caribbean
2,133.247 Ensure full and equal access to modern...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Sexual & reproductive health and rights,- Denmark,- Supported,- Latin America and the Caribbean,- Western Europe & Others
3,133.267 Protect the rights of indigenous peopl...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Right to participate in public affairs & rig...,- Peru,- Supported,- Latin America and the Caribbean,- Latin America and the Caribbean
4,133.268 Continue enhancing the school infrastr...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Right to education,- South Sudan,- Supported,- Latin America and the Caribbean,- Africa
5,133.269 Continue implementing mechanisms for t...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Constitutional & legislative framework,- Cuba,- Supported,- Latin America and the Caribbean,- Latin America and the Caribbean


In [15]:
df2['Themes'] = df2['Themes'].str.split('\n')
df2 = df2.explode('Themes')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1481 entries, 1 to 625
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Text                     1481 non-null   object
 1   Countries Concerned      1481 non-null   object
 2   Document Symbol          1481 non-null   object
 3   Themes                   1481 non-null   object
 4   UPR Recommending States  1481 non-null   object
 5   UPR Position             1481 non-null   object
 6   Regions Concerned        1481 non-null   object
 7   Recommending Regions     1481 non-null   object
dtypes: object(8)
memory usage: 104.1+ KB


In [16]:
df3 = df2.applymap(lambda x: remove_leading_char(str(x)))
df3.head()

,Text,Countries Concerned,Document Symbol,Themes,UPR Recommending States,UPR Position,Regions Concerned,Recommending Regions
Recommendation_id,,,,,,,,
1,Promptly ratify the Optional Protocol to the C...,Uganda,A/HRC/34/10,Labour rights and right to work,Guatemala,Noted,Africa,Latin America and the Caribbean
1,Promptly ratify the Optional Protocol to the C...,Uganda,A/HRC/34/10,Ratification of & accession to international i...,Guatemala,Noted,Africa,Latin America and the Caribbean
1,Promptly ratify the Optional Protocol to the C...,Uganda,A/HRC/34/10,Prohibition of torture & ill-treatment (includ...,Guatemala,Noted,Africa,Latin America and the Caribbean
2,Ensure full and equal access to modern contrac...,Venezuela (Bolivarian Republic of),A/HRC/34/6,Sexual & reproductive health and rights,Denmark,Supported,Latin America and the Caribbean,Western Europe & Others
3,Protect the rights of indigenous peoples throu...,Venezuela (Bolivarian Republic of),A/HRC/34/6,Right to participate in public affairs & right...,Peru,Supported,Latin America and the Caribbean,Latin America and the Caribbean


In [17]:
df4 = df3
df4['Recommending Regions'] = df4['Recommending Regions'].str.split('\n')
df4 = df4.explode('Recommending Regions')
df4 = df4.applymap(lambda x: remove_leading_char(str(x)))
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1539 entries, 1 to 625
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Text                     1539 non-null   object
 1   Countries Concerned      1539 non-null   object
 2   Document Symbol          1539 non-null   object
 3   Themes                   1539 non-null   object
 4   UPR Recommending States  1539 non-null   object
 5   UPR Position             1539 non-null   object
 6   Regions Concerned        1539 non-null   object
 7   Recommending Regions     1539 non-null   object
dtypes: object(8)
memory usage: 108.2+ KB


In [18]:
Countries_df = pd.DataFrame(df4['Countries Concerned'].unique(), columns=['Country_name'])
Countries_df.index = Countries_df.index.set_names(['Country_id'])

In [19]:
Themes_df = pd.DataFrame(df4['Themes'].unique(), columns=['Themes'])
Themes_df.index = Themes_df.index.set_names(['Theme_id'])

In [51]:
Recommendations_df = df[['Text', 'UPR Position']].applymap(lambda x: remove_leading_char(str(x)))

In [53]:
Regions_df = pd.DataFrame(df4['Regions Concerned'].unique(), columns=['Regions'])
Regions_df.index = Regions_df.index.set_names(['Region_id'])